<a href="https://colab.research.google.com/github/larissavaladao/time_series_curuai/blob/main/5.%20parameters_by_period/4.chirps_time_series.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import packages and set up

In [1]:
import ee
import geemap
import pandas as pd

In [2]:
ee.Authenticate()
ee.Initialize(project='ee-curuai2')

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Import Chips dataset

Climate Hazards Center InfraRed Precipitation with Station data (CHIRPS) is a 30+ year quasi-global rainfall dataset. CHIRPS incorporates 0.05° resolution satellite imagery with in-situ station data to create gridded rainfall time series for trend analysis and seasonal drought monitoring.

Resolution
5566 meters

In [4]:
chirps = ee.ImageCollection("UCSB-CHG/CHIRPS/DAILY")


import region of interest

In [5]:
floodplain = ee.FeatureCollection('projects/ee-curuai2/assets/lim_varzea')
floodplain

# filter collection

In [6]:
# //Definir o intervalo de tempo
startyear = 2000
endyear = 2024

In [7]:
# //Definir nossa data inicial e final
startdate = ee.Date.fromYMD(startyear,1,1);
enddate = ee.Date.fromYMD(endyear,12,31);

In [8]:
# //filterando a coleção
chirps = chirps.select('precipitation').filterDate(startdate, enddate).filterBounds(floodplain)
print('qtds',chirps.size().getInfo())

qtds 9131


In [9]:
scale = chirps.first().projection().nominalScale()

# Accumulate precipitation by water period - cota obidos

In [10]:
#import period dates
df_period_limits = pd.read_csv('/content/drive/MyDrive/CURUAI_PROCESS/water_period_limits.csv').drop(columns=['Unnamed: 0'])
# df_period_limits['Data'] = pd.to_datetime(df_period_limits['Data'])
df_period_limits

,Data,type,day,month,year,longitude,latitude
0,2000-01-13 10:43:38.181818240,LW_to_R,13,1,2000,-55.5131,-1.9192
1,2000-02-29 18:17:42.295081984,R_to_HW,29,2,2000,-55.5131,-1.9192
2,2000-08-24 20:45:42.857142784,HW_to_F,24,8,2000,-55.5131,-1.9192
3,2000-09-28 10:23:44.598930432,F_to_LW,28,9,2000,-55.5131,-1.9192
4,2001-01-05 16:27:54.418604672,LW_to_R,5,1,2001,-55.5131,-1.9192
...,...,...,...,...,...,...,...
95,2023-08-25 18:37:31.737451520,F_to_LW,25,8,2023,-55.5131,-1.9192
96,2024-01-26 18:57:28.739495680,LW_to_R,26,1,2024,-55.5131,-1.9192
97,2024-03-25 23:32:02.330097152,R_to_HW,25,3,2024,-55.5131,-1.9192
98,2024-04-06 10:42:51.428571648,HW_to_F,6,4,2024,-55.5131,-1.9192


In [11]:
list_images = []
for i in range(0,len(df_period_limits)-1):
  # print(df_period_limits['Data'][i][0:10])
  # print(df_period_limits['Data'][i+1][0:10])

  image = (chirps.filter(ee.Filter.date(df_period_limits['Data'][i][0:10],df_period_limits['Data'][i+1][0:10]))
              .sum())
  if int(image.bandNames().length().getInfo()) > 0:
    list_images.append(image
              .set('year', str(df_period_limits['year'][i]))
              .set('month_init', str(df_period_limits['month'][i]))
              .set('month_end', str(df_period_limits['month'][i+1]))
              .set('system:time_start', ee.Date.parse('yyyy-MM-dd',str(df_period_limits['Data'][i][0:10])))
              .set('time_start', str(df_period_limits['Data'][i][0:10]))
              .set('time_finish', str(df_period_limits['Data'][i+1][0:10])))



len(list_images)

99

In [12]:
list_images.append(chirps.filterDate('2024-05-30','2025-01-01').sum()
              .set('year', str(2024))
              .set('month_init', str('05'))
              .set('month_end', str(12))
              .set('system:time_start', ee.Date.parse('yyyy-MM-dd',str('2024-05-30')))
              .set('time_start', str('2024-05-30'))
              .set('time_finish', str('2025-01-01')))

In [13]:

period_mosaics =  ee.ImageCollection.fromImages(ee.List(list_images))
period_mosaics.size()

# Accumulate precipitation by water period - month

In [ ]:
# //Criar uma lista sequencial para ano
years = ee.List.sequence(startyear,endyear);

# //Criar uma lista sequencial para meses
months = ee.List([1,4,7,10])

In [ ]:
# //Calculo da precipitação de cada periodo da série

precipitacao_acum =  ee.ImageCollection.fromImages(years.map(lambda y : months.map( lambda m : chirps.filter(ee.Filter.calendarRange(y,y,'year')).filter(ee.Filter.calendarRange(m,ee.Number(m).add(2),'month')).sum().set({'year': y, 'month': m,'system:time_start': ee.Date.fromYMD(y, m, 1)}).set('timestamp', ee.Date.fromYMD(y, m, 1).format('yyyy-MM-dd')))).flatten() )

display(precipitacao_acum.limit(10))

In [ ]:
precipitacao_acum.size()

In [ ]:
# //Calculo da precipitação de cada periodo da série - Opção de média diária.... posso fazer outras médias
# precipitacao_med =  ee.ImageCollection.fromImages(years.map(lambda y : months.map( lambda m : chirps.filter(ee.Filter.calendarRange(y,y,'year')).filter(ee.Filter.calendarRange(m,ee.Number(m).add(2),'month')).mean().set({'year': y, 'month': m,'system:time_start': ee.Date.fromYMD(y, m, 1)}))).flatten() )

# display(precipitacao_med.limit(10))

# Obtain mean accumulated precipitation value in the floodplain for each period

In [14]:
def mean_precip(img):
  '''receives a precipitaion image returns mean value within the floodplain limits
  as a property of the input image'''


  floodplain = ee.FeatureCollection('projects/ee-curuai2/assets/lim_varzea').geometry()


  stats = img.select("precipitation").reduceRegion(**{
    'reducer': ee.Reducer.mean(),
    'geometry': floodplain,
    'scale': chirps.first().projection().nominalScale(),
    'maxPixels': 1e10
    })

  return ee.Feature(floodplain).set({'mean_precipitation':stats.get('precipitation')}).copyProperties(img)

# by month

In [ ]:
precip_acum = ee.FeatureCollection(precipitacao_acum.map(mean_precip))
precip_acum.limit(5)

# by cota period

In [15]:
precip_acum_cota = ee.FeatureCollection(period_mosaics.map(mean_precip))
precip_acum_cota.limit(5)


# Export data

In [ ]:
geemap.ee_to_csv(precip_acum, '/content/drive/MyDrive/CURUAI_PROCESS/mean_precip_acum.csv')

In [16]:
geemap.ee_to_csv(precip_acum_cota, '/content/drive/MyDrive/CURUAI_PROCESS/mean_precip_acum_cota.csv')